## **Background Removal using TRACER - State of the Art RGB Salient Object Detector**

Created a forked repo of TRACER, and made changes there itself, to output images with segmentation masks applied to the original images rather than just simple binary masks.

Dataset Generated: [Background Removed Happywhale Dataset](https://www.kaggle.com/adnanpen/background-removed-happywhale-dataset)

Accordingly made changes to code from testing files to remove evaluation part.

Dataset used for background removal: [Cropped 512x512](https://www.kaggle.com/phalanx/whale2-cropped-dataset)

Original TRACER repo: [TRACER](https://github.com/Karel911/TRACER)

My forked TRACER repo: [TRACER](https://github.com/adnan119/TRACER)

Files modified include: 
[trainer.py](https://github.com/adnan119/TRACER/blob/main/trainer.py) 
[dataloader.py](https://github.com/adnan119/TRACER/blob/main/dataloader.py)
[main.py](https://github.com/adnan119/TRACER/blob/main/main.py)

In [1]:
!git clone https://github.com/adnan119/TRACER.git

fatal: destination path 'TRACER' already exists and is not an empty directory.


In [2]:
!wget https://github.com/Karel911/TRACER/releases/download/v1.0/TRACER-Efficient-7.pth

zsh:1: command not found: wget


In [3]:
!mv ./TRACER-Efficient-7.pth ./best_model.pth


mv: rename ./TRACER-Efficient-7.pth to ./best_model.pth: No such file or directory


In [5]:
%cd ./TRACER
!mkdir ./results/
!mkdir ./results/DUTS/
!mkdir ./results/DUTS/TE5_0/

[Errno 2] No such file or directory: './TRACER'
/Users/marenrieker/Documents/GitHub/happy-whale/TRACER-main/TRACER
mkdir: ./results/: File exists
mkdir: ./results/DUTS/: File exists
mkdir: ./results/DUTS/TE5_0/: File exists


In [6]:
!mv ../best_model.pth ./results/DUTS/TE5_0/


mv: ../best_model.pth: No such file or directory


# <h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">Imports </h1>

In [7]:
import os
import gc
import cv2

import math
import copy
import time
import random
import warnings
import shutil
from pathlib import Path

# For data manipulation
import numpy as np
import pandas as pd
from PIL import *

import torch
import torch.nn as nn
import torch.nn.functional as F

**Segmenting the training set**

In [10]:
#segment test images 
!python main.py test --exp_num 0 --arch 5 --img_size 512 --batch_size 16 --dataset "DUTS" --save_map True --data_path ../../input/whale2-cropped-dataset/cropped_train_images/cropped_train_images/

<---- Training Params ---->
Namespace(RFB_aggregated_channel=[32, 64, 128], action='test', arch='5', aug_ver=2, batch_size=16, channels=[24, 40, 112, 320], clipping=2, criterion='API', data_path='../../input/whale2-cropped-dataset/cropped_train_images/cropped_train_images/', dataset='DUTS', denoise=0.93, epochs=100, exp_num='0', frequency_radius=16, gamma=0.1, img_size=512, lr=5e-05, lr_factor=0.1, model_path='results/', multi_gpu=True, num_workers=4, optimizer='Adam', patience=5, save_map=True, scheduler='Reduce', seed=42, weight_decay=0.0001)
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/adv-efficientnet-b5-86493f6b.pth" to /Users/marenrieker/.cache/torch/hub/checkpoints/adv-efficientnet-b5-86493f6b.pth
100%|████████████████████████████████████████| 117M/117M [00:31<00:00, 3.92MB/s]
Loaded pretrained weights for efficientnet-b5
Traceback (most recent call last):
  File "main.py", line 50, in <module>
    main(cfg)
  File "main.py", line 43, in 

In [13]:
#create zipfile 
shutil.make_archive('./seg_img', 'zip', './seg_img')

FileNotFoundError: [Errno 2] No such file or directory: './seg_img'

In [ ]:
!ls

In [ ]:
seg = os.listdir("./seg_img/") # os.listdir("../input/background-removed-happywhale-dataset/seg_img")
print(len(seg))

In [ ]:
img_num = 17009 #sample test image index
img_name = Path(seg[img_num]).stem

In [ ]:
test_dir = '../input/whale2-cropped-dataset/cropped_train_images/cropped_train_images/'
pil_tes = Image.open(test_dir + img_name + ".jpg")
display(pil_tes)

In [ ]:
pil_im = Image.open("./seg_img/" + img_name + ".png") # Image.open("../input/background-removed-happywhale-dataset/seg_img/" + img_name + ".png")
display(pil_im)

In [ ]:
!rm -r './seg_img' #remove the folder containing the segmented images for saving the zipfile